# **MRI Brain Classification**

## Objective
In this notebook, you will train a convolutional neural network (ConvNet) to classify white blood cells into two categories:
1. Scans of brains with tumors (2D Slices)
2. Normal brain scans (2D slices)


We will use a custom dataset for this classification task. The exercise will guide you step-by-step from data loading and preparation to building and training a deep learning model and visualizing the saliency maps of the models.

Dataset source: https://www.kaggle.com/datasets/navoneel/brain-mri-images-for-brain-tumor-detection



In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

## 📚 0. Import Libraries (Nothing to do)

In [ ]:
# Importing necessary libraries
import tensorflow as tf  # TensorFlow for deep learning
import os  # For handling file paths
import numpy as np  # For data manipulation
import matplotlib.pyplot as plt  # For visualizing the dataset

from tensorflow.keras.preprocessing.image import ImageDataGenerator  # For loading and augmenting image data
from sklearn.model_selection import train_test_split  # For splitting custom dataset into train and test


## 👓 1. Download the data (Nothing to do)
You will use a custom dataset of images that contain lymphoblasts and normal white blood cells. Make sure the dataset is organized in two folders: "tumour" and "normal", with images placed in their respective folders.


In [ ]:
# Define dataset paths
base_dir = '/content/drive/MyDrive/small_mri/'  # Update this to the actual path

# Define parameters
IMG_HEIGHT, IMG_WIDTH = 128, 128
BATCH_SIZE = 16
NUM_EPOCHS=3
RAN_SEED=17

# Use ImageDataGenerator for loading and splitting data
datagen = ImageDataGenerator(
    rescale=1.0/255,  # Normalize pixel values
    validation_split=0.5)  # Split 50% of the data for validation

# Creating train and validation datasets
train_data = datagen.flow_from_directory(
    base_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training',
    seed=RAN_SEED
)

validation_data = datagen.flow_from_directory(
    base_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation',
    seed=RAN_SEED
)

print("Training and validation datasets prepared successfully.")


###  1.2 : Visualize the Dataset

Display 10 images from the training dataset to understand the data better.


In [ ]:
# Function to display a few images from the dataset
def visualize_dataset(dataset, num_images=5):
    plt.figure(figsize=(10, 10))
    for i, (image, label) in enumerate(dataset):
        if i >= num_images:
            break
        plt.subplot(1, num_images, i + 1)
        plt.imshow(image[0])  # Image comes as a batch, so we take the first image
        plt.title("Tumour" if label[0] == 1 else "Normal")
        plt.axis("off")
    plt.show()



# Displaying a few images from the training dataset
visualize_dataset(train_data, num_images=5)


## ➡️ 2. Transfer Learning

In this section, you will use a pretrained MobileNet model, leveraging its features similar to the lymphoblasts vs normal white blood cells task.



In [ ]:
import tensorflow
from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.layers import Input
# Load MobileNetV2 pretrained on ImageNet and add custom layers for our binary classification task
input_tensor = Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3))
# create the base pre-trained model on ImageNet with a custom input tensor
base_model = MobileNet(
    input_tensor=input_tensor,
    include_top=False,  # We do not want the top (classification) layers from MobileNet
    weights='imagenet'  # Load weights pretrained on ImageNet
)

# Freeze the base model to use it as a feature extractor
base_model.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)

# let's add a fully-connected layer
x = Dense(128, activation='relu')(x)
# and a classification layer
predictions = Dense(1, activation='sigmoid')(x)

# this is the model we will train
model_mobilenet = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional MobileNet layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model_mobilenet.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Print the model summary
#model_mobilenet.summary()

### 2.1 Train the MobileNet Model (Nothing to do)

You will now train the model using the training and validation datasets.

In [ ]:
# Train the model
history_mobilenet = model_mobilenet.fit(
    train_data,
    validation_data=validation_data,
    epochs=NUM_EPOCHS,  # You can increase the number of epochs based on your needs
    verbose=1
)

### 2.2 Visualize the results

In [ ]:
# Evaluate the model
loss, accuracy = model_mobilenet.evaluate(validation_data)
print(f"Validation Accuracy: {accuracy:.2f}")

# Plot training and validation accuracy over epochs
acc = history_mobilenet.history['accuracy']
val_acc = history_mobilenet.history['val_accuracy']
loss = history_mobilenet.history['loss']
val_loss = history_mobilenet.history['val_loss']

epochs_range = range(len(acc))

plt.figure(figsize=(14, 5))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')

plt.show()

### 2.3 Compare the performance of your models

**Plot the ROC curves and compute the AUC for your trained model**.

In [ ]:
from sklearn.metrics import roc_curve, auc
import numpy as np

# Function to plot ROC curves for multiple models on the same plot
def plot_roc_curves(models, validation_data_list, labels, title="ROC Curve Comparison"):
    plt.figure(figsize=(10, 6))

    # Iterate over each model and its corresponding validation data
    for model, validation_data, label in zip(models, validation_data_list, labels):
        y_true = []
        y_pred = []

        # Get true labels and predicted probabilities
        for images, labels_batch in validation_data:
            y_true.extend(labels_batch)
            preds = model.predict(images)
            y_pred.extend(preds)

            # Break after one full pass (since it's a generator)
            if len(y_true) >= validation_data.samples:
                break

        y_true = np.array(y_true)
        y_pred = np.array(y_pred)

        # Compute False Positive Rate (FPR) and True Positive Rate (TPR)
        fpr, tpr, _ = roc_curve(y_true, y_pred)
        roc_auc = auc(fpr, tpr)

        # Plot the ROC Curve for the current model
        plt.plot(fpr, tpr, lw=2, label=f'{label} (AUC = {roc_auc:.2f})')

    # Plot the random guess line
    plt.plot([0, 1], [0, 1], color='gray', linestyle='--')  # Random guess line

    # Configure plot settings
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(title)
    plt.legend(loc="lower right")
    plt.show()

# Call the function to plot the ROC curve
plot_roc_curves(
    models=[model_mobilenet],
    validation_data_list=[validation_data],
    labels=["MobileNet Model"]
)

## 🔆 3. Class Activation Maps

In this section, you need to use the GradCAM approach (https://arxiv.org/abs/1610.02391) to visualize the activation maps on some test images.
Below, there is a function which implements the GradCAM algorithm. The function needs the image, the trained model and *the name of the last convolutional layer in the model*.  

In [ ]:
import numpy as np
from tensorflow.keras.models import Model

def get_img_array(img, size):

    #array = tf.keras.preprocessing.image.img_to_array(img)
    # We add a dimension to transform our array into a "batch"
    # of size (1, 299, 299, 3)
    img=np.resize(img,(size,size,3)) #.astype(np.float32)
    array = np.expand_dims(img, axis=0)
    return array


def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    # First, we create a model that maps the input image to the activations
    # of the last conv layer as well as the output predictions
    last_conv_layer = model.get_layer(last_conv_layer_name)
    grad_model = Model(inputs=model.input, outputs=[last_conv_layer.output, model.output])

    # Then, we compute the gradient of the top predicted class for our input image
    # with respect to the activations of the last conv layer
    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        #class_channel = preds[:, pred_index]
        class_channel = preds #[:, 0]
    # This is the gradient of the output neuron (top predicted or chosen)
    # with regard to the output feature map of the last conv layer
    grads = tape.gradient(class_channel, last_conv_layer_output)

    # This is a vector where each entry is the mean intensity of the gradient
    # over a specific feature map channel
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    # We multiply each channel in the feature map array
    # by "how important this channel is" with regard to the top predicted class
    # then sum all the channels to obtain the heatmap class activation
    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    # For visualization purpose, we will also normalize the heatmap between 0 & 1
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()



### 3.1 GradCAM on our model

We're going to select 16 images from the test set and apply the GradCAM to each of them. To each image an extra dimension is added so that their shape is (1,128,128,3). Why do we need that that?

❗ You need to call the make_grad_heatmap() function with the right arguments. Hint: look at the model summary to get the names of all the layers.



In [ ]:
import matplotlib.pyplot as plt
img_size=128


image_examples=[]
labels_examples=[]
cam_examples=[]


images, labels=validation_data[1]
  # Prepare image

for layer in model_mobilenet.layers:
    print(layer.name)

  # Iterate over the images and labels in the current batch
for image, label in zip(images, labels):
  image_examples.append(image)
  labels_examples.append(label)

  image_converted = tf.image.convert_image_dtype(image, tf.float32)
  # Resize image
  image_resized = tf.image.resize(image_converted, (IMG_HEIGHT, IMG_WIDTH))
  img_array=tf.expand_dims(image_resized,axis=0)

  # Make model
  model = model_mobilenet

  # Print what the top predicted class is
  preds = model.predict(img_array)
  print("Label: ", label, "Predicted:", preds[0]) #decode_predictions(preds, top=1)[0])


  heatmap = ### write your code here ... ###

  cam_examples.append(heatmap)

# Display last heatmap
plt.matshow(heatmap)
plt.show()

The size of the CAMs is quite small.
You need to rescale them to the original image size and then superimpose them to the corresponding image. The *display_gradcam* function below creates a transparent heatmap corresponding to the CAM and adds it to the image.

❗You need to modify this function so that the size of the heatmap equals the size of the image. Easy.
❗Equally you need to create a blended image (weighted sum) of the original image and the CAM (use the alpha value to control  transparency)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from matplotlib import cm

def display_gradcam_with_colorbar(img, heatmap, alpha=0.4):
    """
    Superimposes the Grad-CAM heatmap onto the original image and adds a colorbar.

    Args:
    - img: Original image as a NumPy array (H, W, C).
    - heatmap: Grad-CAM heatmap as a NumPy array (H, W).
    - alpha: Intensity factor for the heatmap overlay.

    Returns:
    - superimposed_img: Image with heatmap overlay as a PIL Image.
    """
    # Ensure the image is a NumPy array and normalize it to 0-255
    img = tf.keras.preprocessing.image.img_to_array(img)
    img = (img - img.min()) / (img.max() - img.min())  # Normalize to 0-1
    img = np.uint8(img * 255)  # Scale to 0-255

    # Rescale heatmap to a range of 0-255
    heatmap = np.uint8(255 * heatmap)

    # Use jet colormap to colorize the heatmap
    jet = cm.get_cmap("jet")
    jet_colors = jet(np.arange(256))[:, :3]  # RGB values for the colormap
    jet_heatmap = jet_colors[heatmap]

    # Create an image with RGB heatmap
    jet_heatmap = tf.keras.utils.array_to_img(jet_heatmap)
    jet_heatmap = ### write your code here...
    jet_heatmap = tf.keras.preprocessing.image.img_to_array(jet_heatmap)

    # Superimpose the heatmap on the original image
    superimposed_img = ### write your code here ... ###
    superimposed_img = tf.keras.utils.array_to_img(superimposed_img)

    return superimposed_img

# Visualize Grad-CAMs for multiple examples
def visualize_gradcams_with_colorbars(image_examples, cam_examples, labels_examples, class_names, rows=4, cols=4):
    """
    Visualizes Grad-CAM heatmaps overlaid on images in a grid format with colorbars.

    Args:
    - image_examples: List or array of input images.
    - cam_examples: List or array of heatmaps corresponding to the images.
    - labels_examples: List or array of true labels for the images.
    - class_names: List of class names corresponding to the labels.
    - rows: Number of rows in the grid.
    - cols: Number of columns in the grid.
    """
    fig, axes = plt.subplots(rows, cols, figsize=(20, 20))

    for j, ax in enumerate(axes.flat):
        if j >= len(image_examples):
            break

        # Display the Grad-CAM superimposed image
        img = display_gradcam_with_colorbar(image_examples[j], cam_examples[j])

        # Show the image without calling plt.show() again
        ax.imshow(img)
        ax.set_title(class_names[int(labels_examples[j])])
        ax.axis('off')
        cbar = fig.colorbar(cm.ScalarMappable(cmap="jet"), ax=ax, orientation="vertical")
        cbar.set_label("Activation Intensity", rotation=270, labelpad=15)


    plt.tight_layout()
    plt.show()

# Example usage:
# Assuming `image_examples`, `cam_examples`, and `labels_examples` are predefined
# and `class_names` contains the class labels (e.g., ['normal', 'tumor'])
visualize_gradcams_with_colorbars(image_examples, cam_examples, labels_examples, class_names=['normal', 'tumor'])


## 🔥 4. Pixel-wise saliency maps

❗ You need to complete the pixel-wise saliency map computation here.   
Have look at this paper : https://arxiv.org/abs/1312.6034
which does a deep dive into the algorithm. Equally, check out the keras visualisation kit (https://github.com/keisen/tf-keras-vis).

In [ ]:
def compute_saliency_map(model, img_array, class_index=None):
    """
    Computes a pixel-wise saliency map using the gradients of the model's output with respect to the input image.

    Args:
    - model: Trained model.
    - img_array: Preprocessed input image as a NumPy array (1, H, W, C).
    - class_index: Index of the class to compute the saliency for (optional).

    Returns:
    - saliency_map: Saliency map as a NumPy array (H, W).
    """
    # Ensure the input image is wrapped in a batch
    if len(img_array.shape) == 3:
        img_array = np.expand_dims(img_array, axis=0)

    # Use GradientTape to compute gradients
    with tf.GradientTape() as tape:
        # Watch the input image
        tape.watch(img_array)

        # Forward pass
        predictions = model(img_array)

        # Target the specific class or maximum score
        #if class_index is None:
        #    class_index = tf.argmax(predictions[0])

        target_score = predictions
        #target_score = predictions[:, class_index]

    # Compute the gradient of the target score with respect to the input image
    gradients = tape.gradient(target_score, img_array)

    # Take the absolute value of the gradients and reduce to a single channel
    saliency_map = ## ..write your code here ..

    # Normalize the saliency map to the range [0, 1]
    saliency_map = (saliency_map - tf.reduce_min(saliency_map)) / (tf.reduce_max(saliency_map) - tf.reduce_min(saliency_map))

    return saliency_map.numpy()

❗Display the saliency maps on top of the images in a similar way you did for the GradCAMs.

In [ ]:
image_examples=[]
labels_examples=[]
sal_examples=[]


images, labels=validation_data[2]


for image, label in zip(images, labels):
  image_examples.append(image)
  labels_examples.append(label)

  image_converted = tf.image.convert_image_dtype(image, tf.float32)
  # Resize image
  image_resized = tf.image.resize(image_converted, (IMG_HEIGHT, IMG_WIDTH))
  img_array=tf.expand_dims(image_resized,axis=0)


  # Remove last layer's softmax
  #model.layers[-1].activation = None

  # Print what the top predicted class is
  preds = model.predict(img_array)
  print("Label: ", label, "Predicted:", preds[0])#decode_predictions(preds, top=1)[0])


  # Generate saliency maps
  ## write your code here .....

# Display images
### write your code here ....

❗What do you notice?

❗Try out SmoothGrad or Integrated Gradients

❗Try training your model with data augmentation. Do the saliency maps change?